This notebook is to build a very elementary neural network, trying to predict the expression of certain genes based on DNA sequences in non-coding regions.

In [1]:
import os
from Bio import SeqIO
import numpy as np
import tensorflow as tf

//anaconda/envs/ipykernel_py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Data Processing
We first perform the **one hot encoding** to translate the DNA based "AGCT" into corresponding 0/1 values. One thing to note is that there does exist 'n's in lots of DNA sequences, and we treat them as all false.

In [2]:
base_pairs = {'A': [1, 0, 0, 0], 
'C': [0, 1, 0, 0],
'G': [0, 0, 1, 0],
'T': [0, 0, 0, 1],
'a': [1, 0, 0, 0],
'c': [0, 1, 0, 0],
'g': [0, 0, 1, 0],
't': [0, 0, 0, 1],
'n': [0, 0, 0, 0],
'N': [0, 0, 0, 0]}

Following are some functions to get the one hot encoded DNA data into some input matrix that can be fed into neural network algorithms. The major things to note are the following:
1. DNA sequences are of difference lengths, some very short (100~ bases), some very long (3000~ bases). Since most sequences are in the length range 1000 - 2000, we decide to only take the first 1000 bases of each sequence to train the neural network and make the predictions. If too long, simply truncate it to length 1000. If too short, simply fill with zeros to extend it. 
2. DNA sequences are in different strands, some in negative strand, some in positive. We take the complement of the sequence if it is taken form the negative strand so thsat all our data is from the same (positive) strand.
3. The entire sequence is *flattend*. For example, AGCT would be transformed into [1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1] where the first four represent A and the next four represent G and so on.

In [3]:
def align_sequence(length, sequence):
    if len(sequence) > length:
        aligned_seq = sequence[:length]
    else:
        aligned_seq = sequence + [0]*(length-len(sequence))
    return aligned_seq

In [4]:
# NOT used
def to_positive_strand(strand, sequence):
    if strand == '-':
        unflattened_seq = [base_pairs[n] for n in sequence.complement()]
    else:
        unflattened_seq = [base_pairs[n] for n in sequence]

In [5]:
def process_seq_record(seq_record, X, y):
    header = seq_record.description.split('|')
    expressed = int(header[1])
    y.append(expressed)
    # unflattened_seq = to_positive_strand(header[3], seq_record.seq)
    # NO NEED to reverse complement
    unflattened_seq = [base_pairs[n] for n in seq_record.seq]
    flattened_seq = [i for x in unflattened_seq for i in x]
    aligned_seq = align_sequence(4000, flattened_seq)
    X.append(np.array(aligned_seq))

In [11]:
def read_file(file, X, y):
    seq_record_list = list(SeqIO.parse("../data/input/3.24_species_only/" + file,"fasta"))
    for i in range(len(seq_record_list)):
        process_seq_record(seq_record_list[i], X, y)

Training size is the number of files to read for training. Read 100 files would give us 2400 sequences. <br/> For this simple model, we use 2400 sequence to train the neural network and 240 sequences to test its performance.

In [12]:
training_size = 500
test_size = 50
file_count = 0

X_train = []
y_train = []
X_test = []
y_test = []

for file in os.listdir("../data/input/3.24_species_only"):
    if file.endswith(".fa"):
        if (file_count < training_size):
            read_file(file, X_train, y_train)
        elif (file_count < training_size + test_size):
            read_file(file, X_test, y_test)
        file_count += 1

We examine the shape of all the training and test data matrix to check that the above code works as we expected.

In [13]:
X_train = np.array(X_train)
y_train = np.array(y_train)
if len(y_train.shape) == 1:
    y_train = np.transpose(np.array([y_train]))
X_test = np.array(X_test)
y_test = np.transpose(np.array(y_test))
if len(y_test.shape) == 1:
    y_test = np.transpose(np.array([y_test]))
[X_train.shape, y_train.shape, X_test.shape, y_test.shape]

[(12000, 4000), (12000, 1), (1200, 4000), (1200, 1)]

### Logistic Regression
Before actually getting into the neural network, we first try to implement a very simple logistic regression model to get a taste of the prediction procedure.

In [73]:
from sklearn import linear_model as lm

In [74]:
model = lm.LogisticRegression()
model.fit(X_train, y_train.ravel())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [75]:
y_predicted = np.array(model.predict(X_test))
round(sum(y_test.ravel() == y_predicted)/y_test.shape[0], 3)

0.593

This result of the rate of  correct prediction is slightly better, if any, than random guessing. This suggests that a lot of work needs to be done before we get a satisfying neural network.

### Neural Network with Keras
Now that we have the data ready in the desired numpy array format with correct shapes, we can proceed to train the neural network with our training data in keras and use our test data to see how accurate it performs.

The following cell implements a sequential neural network with 1 input layer (4000 neurons), 4 hidden layers (1000, 400, 40, 10 neurons repectively) and 1 output layer with keras. <br/>
All layers except the final use **relu** as activation functions while the final layer uses **sigmoid** as activation function. The cost fucntion is just defined as the binary crossentrophy. <br/> We train our neural network with 12000 DNA sequences in our training data set.

In [76]:
from keras.layers import Input, Dense
from keras.models import Model, Sequential


model = Sequential()

model.add(Dense(units=1000, activation='relu', input_dim=4000))
model.add(Dense(units=400, activation='relu'))
model.add(Dense(units=40, activation='relu'))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='SGD',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=100, epochs=10)

Epoch 1/10
12000/12000 [==============================] - 8s 644us/step - loss: 0.6742 - acc: 0.5971
Epoch 2/10
12000/12000 [==============================] - 8s 683us/step - loss: 0.6556 - acc: 0.6106
Epoch 3/10
12000/12000 [==============================] - 8s 650us/step - loss: 0.6359 - acc: 0.6281
Epoch 4/10
12000/12000 [==============================] - 9s 723us/step - loss: 0.6100 - acc: 0.6623
Epoch 5/10
12000/12000 [==============================] - 8s 706us/step - loss: 0.5754 - acc: 0.7017
Epoch 6/10
12000/12000 [==============================] - 8s 678us/step - loss: 0.5328 - acc: 0.7380
Epoch 7/10
12000/12000 [==============================] - 7s 611us/step - loss: 0.4815 - acc: 0.7786
Epoch 8/10
12000/12000 [==============================] - 7s 609us/step - loss: 0.4365 - acc: 0.8052
Epoch 9/10
12000/12000 [==============================] - 7s 619us/step - loss: 0.4067 - acc: 0.8159
Epoch 10/10
12000/12000 [==============================] - 7s 623us/step - loss: 0.3816 - a

We can see the accuracy on the training set reaches 83% after 10 epoches. <br/>
Now we test the performance of this model with our test data. <br/> The test data contains 1200 DNA sequences. This size, I think, is already big enough for us to believe that the performance on this test data is quite representative of how the model would perform on new data in general (in other words, with little bias). 

In [77]:
result = model.predict(X_test)

In [78]:
correct = list(np.apply_along_axis(lambda x: 0 if x<0.5 else 1, 1, result))==y_test.ravel()

In [81]:
round(sum(correct)/y_test.shape[0], 3)

0.618

The result is roughly 62%, clearly better than random guessing, which at least suggests that the neural network is actually running. <br/>
However, it is still far from what would be considered a satisfying prediction algorithm.

Some major points to consider that may be helpful to improving the performance of the neural network:
1. Use more complex neural network structure rather than the simple sequential model used above.
2. Incorporate other information like what region of genome is the sequence located, the mapping of transcription factors, etc.
3. Improve the way sequences of different lengths are aligned (Our current approach is truncating the long, filling zero with the short, which probably is too naive and causes significant loss of information)

1. Whole data set OR NOT?
Time changes as data size increases
(data size increases & more than 1000 bases)
2. 